# Signal models
Here we are going to have a look at different signal models available in MRpro.

## Overview

In this notebook we are going to explore the signal models which are implemented in MRpro.

Run this notebook in Google colab: 

<a target="_blank" href="https://colab.research.google.com/github/PTB-MR/mrpro_utrecht_workshop_2024/blob/main/solution_signal_models.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Install MRpro

In [ ]:
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
# Install MRpro from a separate branch including the EPG signal model
!pip install git+https://github.com/PTB-MR/mrpro.git@epg#egg=mrpro[notebook]
import mrpro

## Inversion-recovery model

Let's start with something very simple: a mono-exponential signal recovery after an inversion pulse. 

In [ ]:
# Define the time points after the inversion pulse
import torch
ti = torch.arange(0.1, 10, 0.1)
inversion_recovery_model = mrpro.operators.models.InversionRecovery(ti)


Now we can evaluate it for a couple of $T_1$ times.

In [ ]:
inversion_recovery_signal = inversion_recovery_model.forward(m0=torch.ones(4), t1=torch.as_tensor([1.0, 2.0, 4.0, 8.0]))[0]

import matplotlib.pyplot as plt
for i in range(4):
    plt.plot(ti, inversion_recovery_signal[:,i]);

Often when we want to use this model to fit reconstructed images in Dicom format which correspond to the absolute values of the obtained signals. In MRpro we can simply combine the signal model with a magnitude-operator to create this "new" signal model.

In [ ]:
magn_inversion_recovery_model = mrpro.operators.MagnitudeOp() @ mrpro.operators.models.InversionRecovery(ti)
magn_inversion_recovery_signal = magn_inversion_recovery_model.forward(torch.ones(4), torch.as_tensor([1.0, 2.0, 4.0, 8.0]))[0]

import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,2, squeeze=False)
for i in range(4):
    ax[0,0].plot(ti, inversion_recovery_signal[:,i])
    ax[0,1].plot(ti, magn_inversion_recovery_signal[:,i]);

## Simply monoexponential signal model

Let's try another simple signal model: a monoexponential decay of the signal. This signal model can be e.g. used to describe the $T_2$ decay during a turbo-spin echo acquisition. 

In [ ]:
time_points_during_decay = torch.arange(0,0.5,0.01)
monoexp_model = mrpro.operators.models.MonoExponentialDecay(time_points_during_decay)

Now we can calculate the signal. Here we will only use a single relaxation time but of course multi-dimensional tensors can be provided to the model.

In [ ]:
monoexp_signal = monoexp_model.forward(m0=torch.ones(1), decay_constant=torch.ones(1)*0.1)[0]

plt.figure()
plt.plot(time_points_during_decay, monoexp_signal);

## Extended phase graphs

We also have more advanced models in MRpro, e.g. using the extended phase graph approach. We have a TSE signal model which is decribed by the flip angle and phase of the refocusing pulses, echo times and repetition time between two TSE trains. 

We simulate a TSE train with 20 echoes and an echo time of 10ms. We can either use tensors to define these parameters or single values. If e.g. the flip angle of the refocusing pulse is a tensor of shape (20,) and the phase of the refocusing pulses and the echo time are floats or of shape (1,) then the model assumes that the phase and the echo time are the same for all 20 refocusing pulses.

Keep in mind that we use SI units in MRpro so the angles have to be provided in rad.

In [ ]:
tse_epg_model = mrpro.operators.models.EpgTse(torch.as_tensor([torch.pi,]*20), 0, 0.01)
tse_epg_signal = tse_epg_model.forward(m0=torch.ones(1), t1=torch.ones(1), t2=torch.ones(1)*0.1, b1_scaling_factor=torch.ones(1))[0]

# EPG signals are always complex valued so we have to convert it to real-valued signals
tse_epg_signal = torch.real(tse_epg_signal)

plt.figure()
plt.plot(time_points_during_decay, monoexp_signal)
plt.plot(torch.arange(0.01, 0.01*21, 0.01), tse_epg_signal, 'r-+');

With the extended phase graph approach we have a lot more flexibility than with the mono-exponential decay. We can e.g. also look at signals which are created by refocusing pulses smaller than 180°. 

In [ ]:
tse_epg_model = mrpro.operators.models.EpgTse(torch.as_tensor([torch.pi*0.7,]*20), 0, 0.01)
tse_epg_signal = tse_epg_model.forward(m0=torch.ones(1), t1=torch.ones(1), t2=torch.ones(1)*0.1, b1_scaling_factor=torch.ones(1))[0]

# EPG signals are always complex valued so we have to convert it to real-valued signals
tse_epg_signal = torch.real(tse_epg_signal)

plt.figure()
plt.plot(time_points_during_decay, monoexp_signal)
plt.plot(torch.arange(0.01, 0.01*21, 0.01), tse_epg_signal, 'r-+');